# Import Packages

In [1]:
import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
import workers
from tqdm.notebook import tqdm
import time
from multiprocessing import Pool, cpu_count

# Forecast using Prophet on Item Level

In [2]:
# Import data
submission = pd.read_csv(f'{utils.get_m5_root_dir()}/data/input/sample_submission.csv')
df_calendar, df_sales, df_prices =utils.import_m5_data(reduce_memory=False)

In [3]:
# Has been copied to seperate .py file otherwise it gets stuck in an infinitive loop. 
# Jupyter notebooks cannot cope with multiprocessing


"""
def worker_prophet(df_prophet):
    model = Prophet(daily_seasonality=True, yearly_seasonality=True) 
    model.fit(df_prophet)
    future = model.make_future_dataframe(periods=28, include_history=False)
    forecast = model.predict(future)['yhat'].to_list()
    return forecast

"""
    

"\ndef worker_prophet(df_prophet):\n    model = Prophet(daily_seasonality=True, yearly_seasonality=True) \n    model.fit(df_prophet)\n    future = model.make_future_dataframe(periods=28, include_history=False)\n    forecast = model.predict(future)['yhat'].to_list()\n    return forecast\n\n"

In [4]:
# brigns a pd.Series to a DataFrame of prophet's required structure
def create_prophet_df(pd_series):
    d_string = [f'd_{di}' for di in list(range(1,1914))]
    ds =pd.date_range(start='2011-01-29', end='2016-04-24')
    df = pd.DataFrame({
        'ds': ds,
        'y': pd_series[d_string].values
    })
    return df




In [5]:
# Recude size of data --- only for debugging
df_sales = df_sales.head(200)
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,HOBBIES_1_202_CA_1_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_1,CA,2,0,0,12,...,0,0,0,0,0,0,4,0,4,0
196,HOBBIES_1_203_CA_1_validation,HOBBIES_1_203,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,2,1,0,1,1,1,0,1
197,HOBBIES_1_204_CA_1_validation,HOBBIES_1_204,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
198,HOBBIES_1_205_CA_1_validation,HOBBIES_1_205,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
# Iteratively compute forecasts on item level. 

if __name__ ==  '__main__':

    start = time.time()
    
    # create a list of dataframes containing the sales figures of one item
    list_of_df_sales  = [create_prophet_df(row) for i,row in df_sales.iterrows()]

    # initiate a pool of workers
    pool = Pool(cpu_count())

        
    # paralell computation of forecats
    # list of lists containing the forecasts
    results = pool.map(workers.worker_prophet, tqdm(list_of_df_sales))

    # tranform list of lists to a dataframe
    results_df =pd.DataFrame(results, columns= [f'F{di}' for di in list(range(1,29))])

    # add columns with item's ids    
    results_df.insert(0,'id',df_sales['id'])

    # print computation stats
    print(f'Computation took {int(time.time() - start)} minutes in total.')
    print(f'Computation took {np.round((time.time()-start)/df_sales.shape[0],2)} seconds per item.')
    
    

    



Computation took 111 minutes in total.
Computation took 0.56 seconds per item.


In [7]:
results_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.765524,0.825636,0.709930,0.700229,0.752582,0.899714,0.776378,0.724902,0.783688,...,0.655437,0.802138,0.679068,0.628654,0.689385,0.575150,0.567878,0.623709,0.775442,0.657890
1,HOBBIES_1_002_CA_1_validation,0.148058,0.159780,0.177024,0.228968,0.256769,0.347268,0.289613,0.236834,0.256940,...,0.430566,0.507257,0.432560,0.359923,0.357854,0.357690,0.389008,0.393453,0.458433,0.373722
2,HOBBIES_1_003_CA_1_validation,0.646831,0.656270,0.654694,0.667836,0.706775,0.782454,0.694020,0.641173,0.652129,...,0.733811,0.814480,0.730675,0.681940,0.696344,0.699444,0.716805,0.759348,0.837879,0.751429
3,HOBBIES_1_004_CA_1_validation,1.567754,1.245377,1.184262,1.593367,1.590055,2.680505,3.289942,1.549145,1.237448,...,1.690509,2.791603,3.408682,1.672184,1.361176,1.308076,1.721464,1.718398,2.804839,3.405891
4,HOBBIES_1_005_CA_1_validation,0.682394,0.602300,0.701623,0.633248,0.705551,1.146410,1.055758,0.533515,0.468903,...,0.849026,1.358161,1.335692,0.880029,0.878905,1.057580,1.067268,1.214675,1.726050,1.699779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,HOBBIES_1_202_CA_1_validation,1.204091,1.307237,1.015463,0.900614,0.948502,1.743570,1.361577,1.056422,1.148668,...,0.685809,1.502488,1.145876,0.869142,0.992063,0.724941,0.638834,0.718696,1.548051,1.201712
196,HOBBIES_1_203_CA_1_validation,0.523497,0.497588,0.523098,0.563310,0.574180,0.709348,0.610470,0.549915,0.523735,...,0.616310,0.750341,0.650574,0.589454,0.563095,0.588341,0.628547,0.639733,0.775587,0.677796
197,HOBBIES_1_204_CA_1_validation,0.935177,0.900956,0.984154,0.931973,0.978801,1.032857,0.991988,0.983692,0.949884,...,1.063674,1.114292,1.069522,1.056902,1.018393,1.096649,1.038933,1.079696,1.127228,1.079450
198,HOBBIES_1_205_CA_1_validation,0.116577,0.075893,0.083282,0.102186,0.081393,0.094251,0.114926,0.100203,0.063681,...,0.109681,0.129226,0.155962,0.146576,0.114584,0.129973,0.156057,0.141507,0.159574,0.184345


In [8]:
# Bring data to Kaggel's required form (evaluation & validation)
df_submission = results_df.append(
    results_df.replace(to_replace='validation', value='evaluation', regex=True))
filename = utils.get_m5_root_dir() + '/data/submissions/fbprophet_on_item_level.csv'
df_submission.to_csv(filename, index=False)
df_submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.765524,0.825636,0.709930,0.700229,0.752582,0.899714,0.776378,0.724902,0.783688,...,0.655437,0.802138,0.679068,0.628654,0.689385,0.575150,0.567878,0.623709,0.775442,0.657890
1,HOBBIES_1_002_CA_1_validation,0.148058,0.159780,0.177024,0.228968,0.256769,0.347268,0.289613,0.236834,0.256940,...,0.430566,0.507257,0.432560,0.359923,0.357854,0.357690,0.389008,0.393453,0.458433,0.373722
2,HOBBIES_1_003_CA_1_validation,0.646831,0.656270,0.654694,0.667836,0.706775,0.782454,0.694020,0.641173,0.652129,...,0.733811,0.814480,0.730675,0.681940,0.696344,0.699444,0.716805,0.759348,0.837879,0.751429
3,HOBBIES_1_004_CA_1_validation,1.567754,1.245377,1.184262,1.593367,1.590055,2.680505,3.289942,1.549145,1.237448,...,1.690509,2.791603,3.408682,1.672184,1.361176,1.308076,1.721464,1.718398,2.804839,3.405891
4,HOBBIES_1_005_CA_1_validation,0.682394,0.602300,0.701623,0.633248,0.705551,1.146410,1.055758,0.533515,0.468903,...,0.849026,1.358161,1.335692,0.880029,0.878905,1.057580,1.067268,1.214675,1.726050,1.699779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,HOBBIES_1_202_CA_1_evaluation,1.204091,1.307237,1.015463,0.900614,0.948502,1.743570,1.361577,1.056422,1.148668,...,0.685809,1.502488,1.145876,0.869142,0.992063,0.724941,0.638834,0.718696,1.548051,1.201712
196,HOBBIES_1_203_CA_1_evaluation,0.523497,0.497588,0.523098,0.563310,0.574180,0.709348,0.610470,0.549915,0.523735,...,0.616310,0.750341,0.650574,0.589454,0.563095,0.588341,0.628547,0.639733,0.775587,0.677796
197,HOBBIES_1_204_CA_1_evaluation,0.935177,0.900956,0.984154,0.931973,0.978801,1.032857,0.991988,0.983692,0.949884,...,1.063674,1.114292,1.069522,1.056902,1.018393,1.096649,1.038933,1.079696,1.127228,1.079450
198,HOBBIES_1_205_CA_1_evaluation,0.116577,0.075893,0.083282,0.102186,0.081393,0.094251,0.114926,0.100203,0.063681,...,0.109681,0.129226,0.155962,0.146576,0.114584,0.129973,0.156057,0.141507,0.159574,0.184345


In [9]:
# Submit via Kaggle API or with website https://www.kaggle.com/c/m5-forecasting-accuracy/submissions
!kaggle competitions submit -c m5-forecasting-accuracy -f data/submissions/fbprophet_on_item_level.csv -m"fbprophet_on_item_level.csv"


Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/222k [00:00<?, ?B/s]
 36%|###6      | 80.0k/222k [00:00<00:00, 720kB/s]
100%|##########| 222k/222k [00:04<00:00, 56.6kB/s]
